In [1]:
from __future__ import print_function
import nilearn.plotting as nip
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact,fixed
import inspect

class NiWidget:
    def __init__(self,filename,plotthis='dataobj'):
        self.filename = filename
        


    def plot_slices(self, data, x, y, z, Colormap='viridis'):

        fig,axes=plt.subplots(1,3, figsize = (15, 5))
        axes[0].imshow(np.rot90(data[:,y,:]), cmap = Colormap)
        axes[1].imshow(np.rot90(data[x,:,:]), cmap = Colormap)
        axes[2].imshow(np.rot90(data[:,:,z]), cmap = Colormap)
        plt.show()


    def plot_wrapper(self, data, **kwargs):
        self.plotting_func(data, **kwargs)
        plt.show()


    def nifti_plotter(self,plotting_func=None, **kwargs):
        
        self.plotting_func = plotting_func
        #if no plotting function is specified: use default
        if self.plotting_func is None:
            self.data = nib.load(self.filename).dataobj
            kwargs = {'x': (0,self.data.shape[0]-1),
                         'y': (0,self.data.shape[1]-1),
                         'z': (0,self.data.shape[2]-1),
                         'Colormap': ['jet','viridis','gray']
                         }         
            interact(self.plot_slices, data = fixed(self.data), **kwargs)

        else:
            self.data = nib.load(self.filename)
            interact(self.plot_wrapper, data=fixed(self.data), **kwargs)

/Library/anaconda/envs/rbbrainhack/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
test = NiWidget('cc400_roi_atlas.nii')
test.nifti_plotter()

### This works now I believe!

I think this allows you to pass custom arguments but doesn't fail if there are none

What I did was create a tiny wrapper function that "forwards" all the arguments to the plotting function. I also made the plotting_func a `self.` property that is now accessible by subfunctions since they always get passed `self` anyways.

In [3]:
# With kwargs
test = NiWidget('cc400_roi_atlas.nii')
test.nifti_plotter(plotting_func=nip.plot_glass_brain, threshold=(0,400), display_mode=['ortho','xz'])

In [5]:
# without kwargs (basically just a plot)
test = NiWidget('cc400_roi_atlas.nii')
test.nifti_plotter(plotting_func=nip.plot_glass_brain)

In [ ]:
def plot_glass_threshold(stat_map_img,threshold,display_mode):
    nip.plot_glass_brain(stat_map_img, threshold=threshold, display_mode=display_mode)
    plt.show()

fname = 'cc400_roi_atlas.nii'

kwarg_dict = {'stat_map_img': fixed(fname),
            'threshold': (0,400),
             'display_mode': ['ortho','xz']
             }

interact(plot_glass_threshold,**kwarg_dict)

In [ ]:
#right now this does the same as the above
test.nifti_plotter(plot_glass_threshold,**kwarg_dict)